# Disseny del model DWTFormer

Aquest notebook descriu la implementació de l'arquitectura híbrida DWTFormer, que combina la transformada wavelet discreta (DWT) amb capes Transformer per a la classificació d’imatges mèdiques en el context de prevenció de lesions esportives.

L’objectiu és extreure característiques multiescala mitjançant DWT, i capturar relacions complexes entre regions d’una imatge amb el mecanisme d’atenció dels Transformers.


In [6]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchinfo import summary



Definició del model DWTFormer en PyTorch

In [4]:
class DWTFormer(nn.Module):
    def __init__(self, patch_size=7, num_classes=9, d_model=64, nhead=4, num_layers=2):
        super(DWTFormer, self).__init__()
        self.patch_size = patch_size
        self.num_patches = (14 // patch_size) ** 2
        self.flatten_dim = 4 * patch_size * patch_size  # 4 canals DWT

        self.embedding = nn.Linear(self.flatten_dim, d_model)
        self.positional_encoding = nn.Parameter(torch.randn(1, self.num_patches, d_model))

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, num_classes)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        patches = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        patches = patches.contiguous().view(B, self.num_patches, -1)

        x = self.embedding(patches) + self.positional_encoding
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.classifier(x)


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = DWTFormer()
print(f"Paràmetres totals del model: {count_parameters(model):,}")

x = torch.randn(2, 4, 14, 14)
out = model(x)
print("Forma de sortida:", out.shape)


Paràmetres totals del model: 575,881
Forma de sortida: torch.Size([2, 9])


In [11]:
def inspect_model(model, x):
    with torch.no_grad():
        print("Input:", x.shape)
        patches = x.unfold(2, 7, 7).unfold(3, 7, 7).contiguous().view(x.size(0), -1, 4*7*7)
        print("Patches:", patches.shape)
        emb = model.embedding(patches)
        print("Embedding:", emb.shape)
        trans = model.transformer(emb)
        print("Transformer Output:", trans.shape)
        out = model.classifier(trans.mean(dim=1))
        print("Output:", out.shape)

dummy_input = torch.randn(2, 4, 14, 14)
inspect_model(model, dummy_input)


Input: torch.Size([2, 4, 14, 14])
Patches: torch.Size([2, 4, 196])
Embedding: torch.Size([2, 4, 64])
Transformer Output: torch.Size([2, 4, 64])
Output: torch.Size([2, 9])


## Comparativa d’arquitectures

| Model         | Preprocesament | Capacitat contextual | Cost computacional |
|---------------|----------------|-----------------------|---------------------|
| CNN bàsic     | Cap            | Local                 | Baix                |
| VisionTransformer (ViT) | Patchify       | Global                | Alt                 |
| **DWTFormer** | DWT + Patchify | Global + Multiescala  | Moderat             |

La combinació DWT + Transformer permet capturar tant detalls locals (via subbandes wavelet) com relacions espacials globals (via atenció), essent un bon compromís entre eficiència i rendiment.


## Decisions de disseny

- **Patch size = 7**: divideix perfectament la imatge 14x14 en 4 blocs sense overlapping.
- **4 canals d’entrada**: representen les subbandes LL, LH, HL i HH després de la DWT.
- **Aggregació per mitjana**: simplifica la classificació global mantenint estabilitat.
- **Positional Encoding**: permet mantenir relacions espacials entre els patches.

Aquesta estructura modular facilita provar variants del model i escalar a imatges més grans.
